# Machine Learning 2 - Neural Networks

In this lab, we will use simple Neural Networks to classify the images from the simplified CIFAR-10 dataset. We will compare our results with those obtained with Decision Trees and Random Forests.

Lab objectives
----
* Classification with neural networks
* Influence of hidden layers and of the selected features on the classifier results

In [4]:
from lab_tools import CIFAR10, evaluate_classifier, get_hog_image
        
dataset = CIFAR10('./CIFAR10')

Pre-loading training data
Pre-loading test data


We will use the *[Multi-Layer Perceptron](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)* implementation from scikit-learn, which is only available since version 0.18. You can check which version of scikit-learn is installed by executing this :

In [5]:
import sklearn
print(sklearn.__version__)

1.4.1.post1


If you have version 0.17 or older, please update your scikit-learn installation (for instance, with the command *pip install scikit-learn==0.19.1* in the terminal or Anaconda prompt)

## Build a simple neural network

* Using the [MLPClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) from scikit-learn, create a neural network with a single hidden layer.
* Train this network on the CIFAR dataset.
* Using cross-validation, try to find the best possible parameters.

In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Normalize the HOG features
scaler = StandardScaler()
scaled_train_hog = scaler.fit_transform(dataset.train['hog'])
scaled_test_hog = scaler.transform(dataset.test['hog'])


# Define the parameter grid for the neural network (logistic regression)
param_grid = {
    'activation': ['logistic'],
    'solver': ['lbfgs', 'adam', 'sgd'],
    'alpha': [0.001, 0.01, 0.1, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Initialize MLPClassifier (logistic regression)
mlp_clf = MLPClassifier(hidden_layer_sizes=(), random_state=42)

# Initialize StratifiedKFold with 5 folds
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV with the classifier, parameter grid, and cross-validation
grid_search = GridSearchCV(mlp_clf, param_grid, cv=kf, scoring='accuracy', n_jobs=-1, verbose=2)

# Train the grid search to find the best combination of hyperparameters
grid_search.fit(scaled_train_hog, dataset.train['labels'])
print("Grid Search Completed.")
# Get the best estimator (classifier) from the grid search
best_mlp_clf = grid_search.best_estimator_
print("Best MLP Classifier (Logistic Regression) Found.")
print("Best Hyperparameters:", grid_search.best_params_)
# Now, you can use this best classifier to make predictions on the test set
mlp_test_preds = best_mlp_clf.predict(scaled_test_hog)
mlp_test_accuracy = accuracy_score(dataset.test['labels'], mlp_test_preds)

# Print the best training accuracy found during the cross-validation
best_training_accuracy = grid_search.best_score_
print("Best Training Accuracy (Cross-Validation Score):", best_training_accuracy)

print("MLP Classifier (Logistic Regression) Predictive Performance (Accuracy) on Test Data:", mlp_test_accuracy)


Fitting 5 folds for each of 36 candidates, totalling 180 fits
Grid Search Completed.
Best MLP Classifier (Logistic Regression) Found.
Best Hyperparameters: {'activation': 'logistic', 'alpha': 1, 'learning_rate': 'constant', 'solver': 'sgd'}
Best Training Accuracy (Cross-Validation Score): 0.7500666666666668
MLP Classifier (Logistic Regression) Predictive Performance (Accuracy) on Test Data: 0.7476666666666667


In [5]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(5)

for train,test in kf.split(dataset.train['hog'], dataset.train['labels']):
    train_x = dataset.train['hog'][train]
    train_y = dataset.train['labels'][train]
    
    test_x = dataset.train['hog'][test]
    test_y = dataset.train['labels'][test]

## Add hidden layers to the network.

Try to change the structure of the network by adding hidden layers. Using cross-validation, try to find the best architecture for your network.

In [6]:
from lab_tools import CIFAR10
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Normalize the HOG features
scaler = StandardScaler()
scaled_train_hog = scaler.fit_transform(dataset.train['hog'])
scaled_test_hog = scaler.transform(dataset.test['hog'])

# Define the MLPClassifier
mlp_clf = MLPClassifier(random_state=42, max_iter=1000, early_stopping=True, n_iter_no_change=20, verbose=2)

# Define the parameter grid for the neural network
param_grid = {
    'hidden_layer_sizes': [(128,), (256,), (512,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['lbfgs', 'adam', 'sgd'],
    'alpha': [0.01 ,0.1, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
}

# Initialize GridSearchCV with the classifier, hyperparameters, and cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(mlp_clf, param_grid, cv=kf, n_jobs=-1, verbose=2, scoring='accuracy')

# Train the grid search to find the best combination of hyperparameters
grid_search.fit(scaled_train_hog, dataset.train['labels'])

# Get the best estimator (classifier) from the grid search
best_mlp_clf = grid_search.best_estimator_

# Now, you can use this best classifier to make predictions on the test set
mlp_test_preds = best_mlp_clf.predict(scaled_test_hog)
mlp_test_accuracy = accuracy_score(dataset.test['labels'], mlp_test_preds)

# Print the best parameters and test accuracy
print("Best Hyperparameters:", grid_search.best_params_)
# Print the best training accuracy found during the cross-validation
best_training_accuracy = grid_search.best_score_
print("Best Training Accuracy (Cross-Validation Score):", best_training_accuracy)
# Print the test accuracy
print("Neural Network Predictive Performance (Accuracy) on Test Data:", mlp_test_accuracy)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best Hyperparameters: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (256,), 'learning_rate': 'constant', 'solver': 'adam'}
Neural Network Predictive Performance (Accuracy) on Test Data: 0.8113333333333334
